<a href="https://colab.research.google.com/github/matiasmazzucchi/Python-Prep/blob/main/KromosYbertollina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#llamar a las siguientes bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv

In [2]:
#URL DE CADA PAGINA (terminados en 'K'es de la pagina Kromos y terminados en 'B' es de la pagina de bertollina)
URL_BASE_K = 'https://www.kromosbicicletas.com.ar/productos/mar/1/venzo'
URL_BASE_B = 'https://www.bertolinabikes.com/search/?q=venzo&mpage=2'

#Obtener el HTML de la página
pedido_obtenidoK = requests.get(URL_BASE_K)
html_obtenidoK = pedido_obtenidoK.text
pedido_obtenidoB = requests.get(URL_BASE_B)
html_obtenidoB = pedido_obtenidoB.text


#Parsear el HTML
soupK = BeautifulSoup(html_obtenidoK, 'html.parser')
soupB = BeautifulSoup(html_obtenidoB, 'html.parser')


In [3]:
# Encontrar todos los títulos y precios
#KROMOS
bicicletasK = soupK.find_all('h2', class_='tt-title')
precioK = soupK.find_all('div', class_="tt-price")
#BERTOLLINA
bicicletasB = soupB.find_all('a', class_='js-item-name')
preciosB = soupB.find_all('span', class_='js-price-display item-price h6')
precios_viejosB = soupB.find_all('span', class_="js-compare-price-display item-price-compare p-none-left-xs p-right-quarter-xs")


In [4]:
#Creating the lists for the next dataframe
#KROMOS
bicicletasKL = []
precioKL = []
#BERTOLLINA
bicicletaBL = []
preciosBL = []
precios_viejosBL = []

In [5]:
#Ciclos for para limpiar los datos y obtener los nombres y bicicletas
#KROMOS
for bicicleta in bicicletasK:
 bicicletasKL.append(bicicleta.get_text(strip=True))

for precios in precioK:
  precioKL.append(precios.get_text(strip=True))

#BERTOLLINA
for bicicletas in bicicletasB:
  bicicletaBL.append(bicicletas.get_text(strip=True))

for precios in preciosB:
  preciosBL.append(precios.get_text(strip=True))


for precio_viejo in precios_viejosB:
  precios_viejosBL.append(precio_viejo.get_text(strip=True))


In [6]:
#Armado del dataframe
#KROMOS DATAFRAME
ListaFinalK = list(zip(bicicletasKL, precioKL))
dfK = pd.DataFrame(ListaFinalK, columns=['Bicicleta','Precio'])

#BERTOLLINA DATAFRAME
ListaFinalB = list(zip(bicicletaBL, preciosBL, precios_viejosBL))
dfB = pd.DataFrame(ListaFinalB, columns=['Bicicleta','Precio', 'Precios Viejos'])

In [11]:
# prompt: eliminar los puntos de la columna de 'Precios' y 'Precios Viejos' de la variable 'dfB'

dfB['Precios Viejos'] = dfB['Precios Viejos'].str.replace('.', '')
dfB['Precio'] = dfB['Precio'].str.replace('.', '')


In [14]:
# prompt: eliminar los puntos de la columna 'Precio' de la variable 'dfK' y tambien eliminar los ultimos dos digitos de la columna 'Precio' del la variable 'dfK'

# Eliminar los puntos de la columna 'Precio' de la variable 'dfK'
dfK['Precio'] = dfK['Precio'].str.replace('.', '')

# Eliminar los últimos dos dígitos de la columna 'Precio' del la variable 'dfK'
dfK['Precio'] = dfK['Precio'].str[:-2]


CREATING A DATAFRAME TO EXPORT DATA TO GOOGLE SHEETS FROM GOOGLE COLAB


In [17]:
#Autenticacion y permisos de uso de la API de google SHEETS
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [18]:
#CREACION de la hoja de caluclo de google sheets
sh = gc.create('Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA')

In [20]:
# prompt: Crear una hoja dentro de mi hoja de calculo 'Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA' llamada Kromos

worksheet = gc.open('Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA').add_worksheet(title="Kromos", rows=100, cols=20)


In [21]:
# prompt: Crear una hoja dentro de mi hoja de calculo 'Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA' llamada Bertollina

worksheet = gc.open('Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA').add_worksheet(title="Bertollina", rows=100, cols=20)


In [22]:
#Agregado del Open de KROMOS
worksheetK = gc.open('Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA').worksheet('Kromos')

In [23]:
# Open our new sheet and add some data and also importing the data to the sheet.(KROMOS)
worksheetK.update([dfK.columns.values.tolist()] + dfK.values.tolist())


{'spreadsheetId': '14uJDTTwzc7ik284eUpeX1IAjTkabLL45X9qKb_Sl2Uk',
 'updatedRange': 'Kromos!A1:B21',
 'updatedRows': 21,
 'updatedColumns': 2,
 'updatedCells': 42}

In [24]:
#Agregado del Open de BERTOLLIN
worksheetB = gc.open('Bicicletas Web scraping Gspread KROMOS AND BERTOLLINA').worksheet('Bertollina')

In [25]:
# Open our new sheet and add some data and also importing the data to the sheet. (BERTOLLINA)
worksheetB.update([dfB.columns.values.tolist()] + dfB.values.tolist())

{'spreadsheetId': '14uJDTTwzc7ik284eUpeX1IAjTkabLL45X9qKb_Sl2Uk',
 'updatedRange': 'Bertollina!A1:C17',
 'updatedRows': 17,
 'updatedColumns': 3,
 'updatedCells': 51}



---



In [ ]:
#ESTE ES EL CODIGO COMPLETO PARA DESCARGAR DIRECTAMENTE EL CSV Y HACERLO CON LOS METODOS VIEJOS
#bicicletasL.insert(0, "Bicicletas")
#precioL.insert(0, "Precio")
#datos = dict(zip(bicicletasL, precioL))
#with open ("datos.csv", "w") as f:
 # w = csv.writer(f)
 # w.writerows(datos.items())


Codigo pseudoautomatizado para extraer datos de Corrientes Bikes
---
El problma que tiene este codigo es el de los precios


In [ ]:
URL_BASEC = 'https://corrientesbikes.com.ar/?s=venzo&post_type=product&dgwt_wcas=1'

# Obtener el HTML de la página
pedido_obtenidoC = requests.get(URL_BASEC)
html_obtenidoC = pedido_obtenidoC.text

# Parsear el HTML
soupC = BeautifulSoup(html_obtenidoC, 'html.parser')

In [ ]:
# Encontrar todos los títulos y precios
bicicletasC = soupC.find_all('h2', class_='woocommerce-loop-product__title')
dato_productoC = soupC.find_all ('span', class_='cuotas')
precioC = soupC.find_all('span', class_="woocommerce-Price-currencySymbol")

spansC = soupC.find_all('span', class_='woocommerce-Price-currencySymbol')

#obtencion de precios y limpieza de los mismos
# Iterar sobre los elementos encontrados
for span in spansC:
    # Verificar si el siguiente hermano del span contiene el precio
    if span.next_sibling:
        precioC = span.next_sibling.strip()+','
        print(precioC)

#los valores de los precios se copian y se pegan a mano porque no entiendo que pasa con el ciclo for que no me extraeos dtos ala lsta que arme preciosL

0,
2.468.126,
2.830.625,
481.125,
1.494.875,
798.625,
784.375,
1.494.875,
921.875,
671.875,
584.375,
697.375,
684.375,


In [ ]:
for bicicleta in bicicletasC:
 print(bicicleta.get_text(strip=True)+',')


Venzo Etna,
Venzo X-Force,
Venzo Loki R29,
Venzo Atix 2×11,
Venzo Raptor EXO,
Venzo Phoenix Sensah,
Venzo Atix Evo 30 Vel.,
Venzo Raptor 3×9 Mecanico,
Venzo Eolo Evo 24V Sensa R29,
Venzo Skyline disco Mecanico,
Venzo Raptor 24 Vel. a Rosca,
Venzo Primal 24 Vel a Cassete,


In [ ]:
for datos_producto in dato_productoC:
  print(datos_producto.get_text(strip=True)+',')

3 cuotas sin interés de  $822.708,67,
3 cuotas sin interés de  $943.541,67,
3 cuotas sin interés de  $160.375,00,
3 cuotas sin interés de  $498.291,67,
3 cuotas sin interés de  $266.208,33,
3 cuotas sin interés de  $261.458,33,
3 cuotas sin interés de  $498.291,67,
3 cuotas sin interés de  $307.291,67,
3 cuotas sin interés de  $223.958,33,
3 cuotas sin interés de  $194.791,67,
3 cuotas sin interés de  $232.458,33,
3 cuotas sin interés de  $228.125,00,


In [ ]:
#Creado de las listas, menos la de precio
bicicletasLC = []
datosLC = []
preciosLC = [0,
2.468.126,
2.830.625,
481.125,
1.494.875,
798.625,
784.375,
1.494.875,
921.875,
671.875,
584.375,
697.375,
684.375,]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-59-b8e3e2998eeb>, line 5)

In [ ]:
#CREAR UN DATAFRAME PARA Corrientes


In [ ]:
#CREAR LA HOJA DE SHEETS para almacenar los datos de Corrientes Bikes Shop
worksheetC = gc.open('Bicicletas Web scraping Gspread').worksheet('coorientes bikes')



CODIGO PARA EXTRAER LOS DATOS DE FACHIN
---



In [ ]:
#introduccir
URL_BASEF = 'https://www.facchinbicycles.com.ar/listado/deportes-fitness/ciclismo/bicicletas/venzo/'

# Obtener el HTML de la página
pedido_obtenidoF = requests.get(URL_BASEF)
html_obtenidoF = pedido_obtenidoF.text

# Parsear el HTML
soupF = BeautifulSoup(html_obtenidoF, 'html.parser')

In [ ]:
bicicletaLF = []
preciosLF = []

In [ ]:

# Encontrar todos los títulos y precios
bicicletaF = soupF.find_all('h2', class_='ui-search-item__title shops__item-title')
preciosF = soupF.find_all('div', class_='ui-search-price ui-search-price--size-medium shops__price')

In [ ]:
for bicicletas in bicicletaF:
  bicicletaLF.append(bicicletas.get_text(strip=True))

In [ ]:
#estos datos van a estar registrados como una lista completa y luego se va a dividir el texto en columnas po $ y % en google sheets
for precio in preciosF:
  preciosLF.append(precio.get_text(strip=True))

In [ ]:
#CREACION DEL DATAFRAME
ListaFinalF = list(zip(bicicletaLF, preciosLF))
dfF = pd.DataFrame(ListaFinalF, columns=['Bicicleta','Precio'])

In [ ]:
#Agregado del Open de BERTOLLIN
worksheetF = gc.open('Bicicletas Web scraping Gspread').worksheet('fachin')

In [ ]:
# Open our new sheet and add some data and also importing the data to the sheet. (BERTOLLINA)
worksheetF.update([dfF.columns.values.tolist()] + dfF.values.tolist())

{'spreadsheetId': '1JFs2g8hMzAK482Gi7pou_h5JjhTDncMTxRbH0ws4bXk',
 'updatedRange': 'fachin!A1:B51',
 'updatedRows': 51,
 'updatedColumns': 2,
 'updatedCells': 102}